In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os

In [4]:
from feature_selection import forwardFeatureSelection

from GenLinCFA import GenLinCFA

from aux_GenLinCFA import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,FS_with_logisticWrapper,compare_methods,compare_methods_class,compute_r2,prepare_target_binary

In [5]:
cd ..

/content/drive/MyDrive/Colab Notebooks


## GenLinCFA aggregations standardized target



In [ ]:
basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './GenLinCFA/temp_prec/internal_ordering/'
#plots_folder = './GenLinCFA/for_plots/internal_ordering/'

for basin in basins:
  selected_colnames_CMI5 = []
  print('####################' + basin + '####################')
  target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv', window_size = 1)
  eps = 0.37
  actual_path = path_features+basin+'_aggreg.csv'
  output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                            'cyclostationary_mean_tg_1w',
                                                                            'cyclostationary_mean_tg_4w',
                                                                            'cyclostationary_mean_tg_8w',
                                                                            'cyclostationary_mean_tg_12w',
                                                                            'cyclostationary_mean_tg_16w',
                                                                            'cyclostationary_mean_tg_24w',
                                                                            'cyclostationary_mean_rr',
                                                                            'cyclostationary_mean_rr_1w',
                                                                            'cyclostationary_mean_rr_4w',
                                                                            'cyclostationary_mean_rr_8w',
                                                                            'cyclostationary_mean_rr_12w',
                                                                            'cyclostationary_mean_rr_16w',
                                                                            'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                      target_df_trainVal, eps=eps,
                                                                      max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

  #agg_trainVal_string = plots_folder + basin + "_trainVal_aggreg"
  #agg_test_string = plots_folder + basin + "_test_aggreg"
  #aggregate_trainVal.to_csv(agg_trainVal_string, index = False)
  #aggregate_test.to_csv(agg_test_string, index = False)

  selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

  print('\nFull model and selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test.csv'
  X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
  X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
  X_train_wrapper.to_csv(train_string, index=False)
  X_validation_wrapper.to_csv(val_string, index=False)
  X_test_wrapper.to_csv(test_string, index=False)

  res = {
          "delta" : [],
          "numSelected" : [],
          "selectedFeatures" : []
      }

  res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                    np.array(target_df_trainVal.mean_std),res,10,1)

  selectedFeatures='selectedFeatures'
  print(f'\n{res[selectedFeatures]}\n')

  selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

  print('\nFull model and selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test.csv'

  X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
  X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

  selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values)

  X_train_CMI5.to_csv(train_string, index=False)
  X_validation_CMI5.to_csv(val_string, index=False)
  X_test_CMI5.to_csv(test_string, index=False)

  train_string = destination_folder + basin + '_genLinCFA_CMI_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_CMI_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_CMI_test.csv'

  X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
  X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
  X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
  X_test_CMI = aggregate_test.loc[:,selected_colnames]

  X_train_CMI.to_csv(train_string, index=False)
  X_validation_CMI.to_csv(val_string, index=False)
  X_test_CMI.to_csv(test_string, index=False)

  print('###### Binary Classification ######')

  target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

  log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

  # CMI
  log_regr.fit(X_train_validation_CMI, target_df_trainVal)
  print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

  # CMI best 5
  log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
  print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

  # wrapper
  log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
  print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
  print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

  #output_string = plots_folder + basin + '_aggregations.npy'
  #sel_col_string = plots_folder + basin + '_chosen_features.npy'
  #np.save(sel_col_string, selected_colnames_CMI5)
  #np.save(output_string, outputs)

####################Emiliani1####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1         0
1    2001-01-13  0.482679    0.58  2001     2         1
2    2001-01-21  0.516259    0.59  2001     3         1
3    2001-01-29  0.434421    0.50  2001     5         0
4    2001-02-06  0.494805    0.54  2001     6         1
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.427085    0.43  2009    48         0
407  2009-12-05  0.547380    0.57  2009    49         1
408  2009-12-13  0.531070    0.58  2009    50         1
409  2009-12-21  0.295704    0.00  2009    52         0
410  2009-12-29  0.027861    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 172

Number of aggregated features: 3

Feature: cyclostationary_mean_tg_1w

Number of features: 172

Number of aggregated featu

In [ ]:
basins = ['Adda', 'Lambro_Olona', 'Ticino', 'Oglio_Iseo']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './GenLinCFA/temp_prec/internal_ordering/'
#plots_folder = './GenLinCFA/for_plots/internal_ordering/'

for basin in basins:
  selected_colnames_CMI5 = []
  outputs = []
  print('####################' + basin + '####################')
  target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv', window_size = 1)
  eps = 0.37
  actual_path = path_features+basin+'_aggreg.csv'
  output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                            'cyclostationary_mean_tg_1w',
                                                                            'cyclostationary_mean_tg_4w',
                                                                            'cyclostationary_mean_tg_8w',
                                                                            'cyclostationary_mean_tg_12w',
                                                                            'cyclostationary_mean_tg_16w',
                                                                            'cyclostationary_mean_tg_24w',
                                                                            'cyclostationary_mean_rr',
                                                                            'cyclostationary_mean_rr_1w',
                                                                            'cyclostationary_mean_rr_4w',
                                                                            'cyclostationary_mean_rr_8w',
                                                                            'cyclostationary_mean_rr_12w',
                                                                            'cyclostationary_mean_rr_16w',
                                                                            'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                      target_df_trainVal, eps=eps,
                                                                      max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

  #agg_trainVal_string = plots_folder + basin + "_trainVal_aggreg"
  #agg_test_string = plots_folder + basin + "_test_aggreg"
  #aggregate_trainVal.to_csv(agg_trainVal_string, index = False)
  #aggregate_test.to_csv(agg_test_string, index = False)

  selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

  print('\nFull model and selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test.csv'
  X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
  X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
  X_train_wrapper.to_csv(train_string, index=False)
  X_validation_wrapper.to_csv(val_string, index=False)
  X_test_wrapper.to_csv(test_string, index=False)

  res = {
          "delta" : [],
          "numSelected" : [],
          "selectedFeatures" : []
      }

  res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                    np.array(target_df_trainVal.mean_std),res,10,1)

  selectedFeatures='selectedFeatures'
  print(f'\n{res[selectedFeatures]}\n')

  selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

  print('\nFull model and selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test.csv'

  X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
  X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

  selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values)

  X_train_CMI5.to_csv(train_string, index=False)
  X_validation_CMI5.to_csv(val_string, index=False)
  X_test_CMI5.to_csv(test_string, index=False)

  train_string = destination_folder + basin + '_genLinCFA_CMI_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_CMI_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_CMI_test.csv'

  X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
  X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
  X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
  X_test_CMI = aggregate_test.loc[:,selected_colnames]

  X_train_CMI.to_csv(train_string, index=False)
  X_validation_CMI.to_csv(val_string, index=False)
  X_test_CMI.to_csv(test_string, index=False)

  print('###### Binary Classification ######')

  target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

  log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

  # CMI
  log_regr.fit(X_train_validation_CMI, target_df_trainVal)
  print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

  # CMI best 5
  log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
  print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

  # wrapper
  log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
  print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
  print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

  #output_string = plots_folder + basin + '_aggregations.npy'
  #sel_col_string = plots_folder + basin + '_chosen_features.npy'
  #np.save(sel_col_string, selected_colnames_CMI5)
  #np.save(output_string, outputs)

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1         0
1    2001-01-13  0.380618    0.43  2001     2         0
2    2001-01-21  0.341985    0.38  2001     3         0
3    2001-01-29  0.322044    0.35  2001     5         0
4    2001-02-06  0.354954    0.40  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48         0
407  2009-12-05  0.409921    0.46  2009    49         0
408  2009-12-13  0.472087    0.53  2009    50         1
409  2009-12-21  0.324728    0.00  2009    52         0
410  2009-12-29  0.086512    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 92

Number of aggregated features: 3

Feature: cyclostationary_mean_tg_1w

Number of features: 92

Number of aggregated features: 3


In [ ]:
basins = ['Dora', 'Piemonte_Sud', 'Piemonte_Nord']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './GenLinCFA/temp_prec/internal_ordering/'
#plots_folder = './GenLinCFA/for_plots/internal_ordering/'

for basin in basins:
  selected_colnames_CMI5 = []
  print('####################' + basin + '####################')
  target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv', window_size = 1)
  eps = 0.37
  actual_path = path_features+basin+'_aggreg.csv'
  output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                            'cyclostationary_mean_tg_1w',
                                                                            'cyclostationary_mean_tg_4w',
                                                                            'cyclostationary_mean_tg_8w',
                                                                            'cyclostationary_mean_tg_12w',
                                                                            'cyclostationary_mean_tg_16w',
                                                                            'cyclostationary_mean_tg_24w',
                                                                            'cyclostationary_mean_rr',
                                                                            'cyclostationary_mean_rr_1w',
                                                                            'cyclostationary_mean_rr_4w',
                                                                            'cyclostationary_mean_rr_8w',
                                                                            'cyclostationary_mean_rr_12w',
                                                                            'cyclostationary_mean_rr_16w',
                                                                            'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                      target_df_trainVal, eps=eps,
                                                                      max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

  #agg_trainVal_string = plots_folder + basin + "_trainVal_aggreg"
  #agg_test_string = plots_folder + basin + "_test_aggreg"
  #aggregate_trainVal.to_csv(agg_trainVal_string, index = False)
  #aggregate_test.to_csv(agg_test_string, index = False)

  selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

  print('\nFull model and selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test.csv'
  X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
  X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
  X_train_wrapper.to_csv(train_string, index=False)
  X_validation_wrapper.to_csv(val_string, index=False)
  X_test_wrapper.to_csv(test_string, index=False)

  res = {
          "delta" : [],
          "numSelected" : [],
          "selectedFeatures" : []
      }

  res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                    np.array(target_df_trainVal.mean_std),res,10,1)

  selectedFeatures='selectedFeatures'
  print(f'\n{res[selectedFeatures]}\n')

  selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

  print('\nFull model and selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test.csv'

  X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
  X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

  selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values)

  X_train_CMI5.to_csv(train_string, index=False)
  X_validation_CMI5.to_csv(val_string, index=False)
  X_test_CMI5.to_csv(test_string, index=False)

  train_string = destination_folder + basin + '_genLinCFA_CMI_train.csv'
  val_string = destination_folder + basin + '_genLinCFA_CMI_val.csv'
  test_string = destination_folder + basin + '_genLinCFA_CMI_test.csv'

  X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
  X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
  X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
  X_test_CMI = aggregate_test.loc[:,selected_colnames]

  X_train_CMI.to_csv(train_string, index=False)
  X_validation_CMI.to_csv(val_string, index=False)
  X_test_CMI.to_csv(test_string, index=False)

  print('###### Binary Classification ######')

  target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

  log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

  # CMI
  log_regr.fit(X_train_validation_CMI, target_df_trainVal)
  print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

  # CMI best 5
  log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
  print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

  # wrapper
  log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
  print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
  print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

  #output_string = plots_folder + basin + '_aggregations.npy'
  #sel_col_string = plots_folder + basin + '_chosen_features.npy'
  #np.save(sel_col_string, selected_colnames_CMI5)
  #np.save(output_string, outputs)

####################Dora####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1         0
1    2001-01-13  0.206769    0.00  2001     2         0
2    2001-01-21  0.267313    0.00  2001     3         0
3    2001-01-29  0.240836    0.20  2001     5         0
4    2001-02-06  0.193417    0.15  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.230073    0.25  2009    48         0
407  2009-12-05  0.243632    0.24  2009    49         0
408  2009-12-13  0.251111    0.00  2009    50         0
409  2009-12-21  0.099246    0.00  2009    52         0
410  2009-12-29  0.064990    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 44

Number of aggregated features: 5

Feature: cyclostationary_mean_tg_1w

Number of features: 44

Number of aggregated features: 5


## Temp Prec Snow

In [ ]:
basins = ['Ticino', 'Adda', 'Oglio_Iseo', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'
path_features='./features_allvalues/'

destination_folder = './GenLinCFA/temp_prec_snow_copernicus/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv')

    eps = 0.37
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'

    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', scale = 0.26)

      aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow,aggregate_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train_withSnow.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val_withSnow.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test_withSnow.csv'
    X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
    X_train_wrapper.to_csv(train_string, index=False)
    X_validation_wrapper.to_csv(val_string, index=False)
    X_test_wrapper.to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')

    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test_withSnow.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5 = aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_genLinCFA_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_genLinCFA_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_genLinCFA_CMI_test_withSnow.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    X_train_CMI.to_csv(train_string, index=False)
    X_validation_CMI.to_csv(val_string, index=False)
    X_test_CMI.to_csv(test_string, index=False)

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    log_regr.fit(X_train_validation_CMI, target_df_trainVal)
    print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    # wrapper
    log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
    print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
    print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

####################Ticino####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.264043    0.00  2001     1         0
1    2001-01-13  0.354618    0.39  2001     2         0
2    2001-01-21  0.427990    0.47  2001     3         1
3    2001-01-29  0.339495    0.35  2001     5         0
4    2001-02-06  0.324134    0.34  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.332713    0.35  2009    48         0
407  2009-12-05  0.370253    0.40  2009    49         0
408  2009-12-13  0.517201    0.57  2009    50         1
409  2009-12-21  0.353636    0.45  2009    52         0
410  2009-12-29  0.261079    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 92

Number of aggregated features: 5

Feature: cyclostationary_mean_tg_1w

Number of features: 92

Number of aggregated features: 

## Temp Prec Snow Lakes

In [ ]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'
path_features='./features_allvalues/'
path_features_lakes='./lakes/'

destination_folder = './GenLinCFA/temp_prec_snow_lakes_copernicus/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv')

    eps = 0.37
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', scale = 0.26)

      aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow,aggregate_test),axis=1)

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)
      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((aggregate_trainVal, df_lakes_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test, df_lakes_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test_withSnowLakes.csv'
    X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
    X_train_wrapper.to_csv(train_string, index=False)
    X_validation_wrapper.to_csv(val_string, index=False)
    X_test_wrapper.to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')

    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test_withSnowLakes.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5 = aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_genLinCFA_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_CMI_test_withSnowLakes.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    X_train_CMI.to_csv(train_string, index=False)
    X_validation_CMI.to_csv(val_string, index=False)
    X_test_CMI.to_csv(test_string, index=False)

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    log_regr.fit(X_train_validation_CMI, target_df_trainVal)
    print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    # wrapper
    log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
    print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
    print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

####################Ticino####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.264043    0.00  2001     1         0
1    2001-01-13  0.354618    0.39  2001     2         0
2    2001-01-21  0.427990    0.47  2001     3         1
3    2001-01-29  0.339495    0.35  2001     5         0
4    2001-02-06  0.324134    0.34  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.332713    0.35  2009    48         0
407  2009-12-05  0.370253    0.40  2009    49         0
408  2009-12-13  0.517201    0.57  2009    50         1
409  2009-12-21  0.353636    0.45  2009    52         0
410  2009-12-29  0.261079    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 92

Number of aggregated features: 5

Feature: cyclostationary_mean_tg_1w

Number of features: 92

Number of aggregated features: 

## Temp Prec Lakes

In [ ]:
basins = ['Adda', 'Ticino', 'Oglio_Iseo', 'Lambro_Olona']
path_target = "./csv/"
path_features='./features_allvalues/'
path_features_lakes='./lakes/'
destination_folder = './GenLinCFA/temp_prec_lakes/internal_ordering/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv', window_size = 1)

    eps = 0.37
    actual_path = path_features+basin+'_aggreg.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)

      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((df_lakes_trainVal,aggregate_trainVal_temp_prec),axis=1)
      aggregate_test = pd.concat((df_lakes_test,aggregate_test_temp_prec),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train_withLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val_withLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test_withLakes.csv'
    X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
    X_train_wrapper.to_csv(train_string, index=False)
    X_validation_wrapper.to_csv(val_string, index=False)
    X_test_wrapper.to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')

    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test_withLakes.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5 = aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_genLinCFA_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_CMI_test_withLakes.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    X_train_CMI.to_csv(train_string, index=False)
    X_validation_CMI.to_csv(val_string, index=False)
    X_test_CMI.to_csv(test_string, index=False)

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    log_regr.fit(X_train_validation_CMI, target_df_trainVal)
    print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    # wrapper
    log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
    print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
    print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1         0
1    2001-01-13  0.380618    0.43  2001     2         0
2    2001-01-21  0.341985    0.38  2001     3         0
3    2001-01-29  0.322044    0.35  2001     5         0
4    2001-02-06  0.354954    0.40  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48         0
407  2009-12-05  0.409921    0.46  2009    49         0
408  2009-12-13  0.472087    0.53  2009    50         1
409  2009-12-21  0.324728    0.00  2009    52         0
410  2009-12-29  0.086512    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Feature: cyclostationary_mean_tg

Number of features: 92

Number of aggregated features: 3

Feature: cyclostationary_mean_tg_1w

Number of features: 92

Number of aggregated features: 3


## Multi task scores

In [6]:
# for binary classification

from sklearn.metrics import accuracy_score
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

In [ ]:
### binary targets
basins = ['Emiliani1','Emiliani2','Garda_Mincio', 'Adda', 'Lambro_Olona', 'Ticino',
          'Oglio_Iseo', 'Dora', 'Piemonte_Sud', 'Piemonte_Nord']

path_target = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv', window_size = 1)

    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

## Temp Prec

In [8]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona',
          'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

### CMI best5 features
path_features = './GenLinCFA/temp_prec/internal_ordering/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [9]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona',
          'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

### CMI features
path_features = './GenLinCFA/temp_prec/internal_ordering/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [10]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona',
          'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

### wrapper best5 features
path_features = './GenLinCFA/temp_prec/internal_ordering/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

## Temp Prec Snow

In [26]:
basins = ['Adda', 'Oglio_Iseo', 'Ticino', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

### CMI best5 features
path_features = './GenLinCFA/temp_prec_snow_copernicus/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test_withSnow.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

# exception for basin with no snow
basin = 'Lambro_Olona'
path_features = './GenLinCFA/temp_prec/internal_ordering/'
train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train.csv')
val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val.csv')
test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test.csv')
best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [27]:
basins = ['Adda', 'Oglio_Iseo', 'Ticino', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

### CMI features
path_features = './GenLinCFA/temp_prec_snow_copernicus/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test_withSnow.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './GenLinCFA/temp_prec/internal_ordering/'
train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train.csv')
val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val.csv')
test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test.csv')
CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [28]:
basins = ['Adda', 'Oglio_Iseo', 'Ticino', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

### wrapper best5 features
path_features = './GenLinCFA/temp_prec_snow_copernicus/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test_withSnow.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './GenLinCFA/temp_prec/internal_ordering/'
train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train.csv')
val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val.csv')
test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test.csv')
best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

## Temp Prec Snow Lakes

In [39]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### CMI best5 features
path_features = './GenLinCFA/temp_prec_snow_lakes_copernicus/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test_withSnowLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [40]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### CMI features
path_features = './GenLinCFA/temp_prec_snow_lakes_copernicus/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test_withSnowLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [41]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### wrapper best5 features
path_features = './GenLinCFA/temp_prec_snow_lakes_copernicus/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test_withSnowLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

## Temp Prec Lakes

In [51]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### CMI best5 features
path_features = './GenLinCFA/temp_prec_lakes/internal_ordering/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test_withLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [52]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### CMI features
path_features = './GenLinCFA/temp_prec_lakes/internal_ordering/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test_withLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [53]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### wrapper best5 features
path_features = './GenLinCFA/temp_prec_lakes/internal_ordering/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test_withLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

## MTL scores

### CMI best 5

In [54]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.5833333333333334
Emiliani2
0.5789473684210527
Garda_Mincio
0.6491228070175439


In [55]:
MTL_scores(clust_basins=['Dora','Piemonte_Sud', 'Piemonte_Nord'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.40350877192982454
Piemonte_Sud
0.5921052631578947
Piemonte_Nord
0.5745614035087719


In [56]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6666666666666666
Lambro_Olona
0.6929824561403509
Oglio_Iseo
0.6710526315789473
Ticino
0.706140350877193


### CMI

In [57]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.5833333333333334
Emiliani2
0.5789473684210527
Garda_Mincio
0.6491228070175439


In [58]:
MTL_scores(clust_basins=['Dora','Piemonte_Sud', 'Piemonte_Nord'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.40350877192982454
Piemonte_Sud
0.5921052631578947
Piemonte_Nord
0.5745614035087719


In [59]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6754385964912281
Lambro_Olona
0.6973684210526315
Oglio_Iseo
0.6578947368421053
Ticino
0.7017543859649122


### Wrapper best 5

In [60]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.5833333333333334
Emiliani2
0.5789473684210527
Garda_Mincio
0.6491228070175439


In [61]:
MTL_scores(clust_basins=['Dora','Piemonte_Sud', 'Piemonte_Nord'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.40350877192982454
Piemonte_Sud
0.5921052631578947
Piemonte_Nord
0.5745614035087719


In [62]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6578947368421053
Lambro_Olona
0.6535087719298246
Oglio_Iseo
0.6754385964912281
Ticino
0.6578947368421053
